In [ ]:
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network._multilayer_perceptron import MLPRegressor
from statsmodels.tsa.arima.model import ARIMA
from google.oauth2 import service_account
from datetime import datetime as date
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.metrics import r2_score,make_scorer,mean_squared_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

import joblib
import pickle
import pandas as pd
import pandas_gbq
import numpy as np
import xgboost
import lightgbm
import shap

### Gathering Data and Feature Engineering

In [ ]:
player_features= ["min_3gm_avg", "fgm_3gm_avg", "fga_3gm_avg", "fg%_3gm_avg", "3pm_3gm_avg", 
                   "3pa_3gm_avg", "3p%_3gm_avg", "ftm_3gm_avg", "fta_3gm_avg", "ft%_3gm_avg", 
                   "oreb_3gm_avg", "dreb_3gm_avg", "reb_3gm_avg", "ast_3gm_avg", "stl_3gm_avg", 
                   "blk_3gm_avg", "to_3gm_avg", "pf_3gm_avg", "pts_3gm_avg", "plus_mins_3gm_avg",
                   "pts_season", "pts_momentum", "min_season", "min_momentum", "fgm_season", "fgm_momentum", 
                    "fga_season", "fga_momentum", "fg%_season", "fg%_momentum", "3pm_season", "3pm_momentum", 
                    "3pa_season", "3pa_momentum", "3p%_season", "3p%_momentum", "ftm_season", "ftm_momentum", 
                    "fta_season", "fta_momentum", "ft%_season", "ft%_momentum", "oreb_season", "oreb_momentum", 
                    "dreb_season", "dreb_momentum", "reb_season", "reb_momentum", "ast_season", "ast_momentum", 
                    "stl_season", "stl_momentum", "blk_season", "blk_momentum", "to_season", "to_momentum", 
                    "pf_season", "pf_momentum", "plus_mins_season", "plus_mins_momentum"]

team_features = ["home","away","offrtg_3gm_avg", "defrtg_3gm_avg", "netrtg_3gm_avg", "ast%_3gm_avg", "ast_to_3gm_avg", 
                    "ast_ratio_3gm_avg", "oreb%_3gm_avg", "dreb%_3gm_avg", "reb%_3gm_avg", "tov%_3gm_avg", 
                    "efg%_3gm_avg", "ts%_3gm_avg", "pace_3gm_avg", "pie_3gm_avg",
                    "netrtg_season", "netrtg_momentum", "offrtg_season", "offrtg_momentum", "defrtg_season", "defrtg_momentum", 
                    "ast%_season", "ast%_momentum", "ast_to_season", "ast_to_momentum", "ast_ratio_season", "ast_ratio_momentum", 
                    "oreb%_season", "oreb%_momentum", "dreb%_season", "dreb%_momentum", "reb%_season", "reb%_momentum", 
                    "tov%_season", "tov%_momentum", "efg%_season", "efg%_momentum", "ts%_season", "ts%_momentum", 
                    "pace_season", "pace_momentum", "pie_season", "pie_momentum"]

#using shifted windows for rolling data to prevent data leakage
player_query = f""" 
SELECT player,team,game_id,game_date,matchup,pts,reb,ast,`3pm`, {','.join([f'`{player}`' for player in player_features])},season
from `capstone_data.player_modeling_data`
order by game_date asc
"""

team_query = f"""
SELECT team,game_id,game_date,home,away, {', '.join([f'`{team}`' for team in team_features])}
from `capstone_data.team_modeling_data`
order by game_date asc
"""


In [ ]:
try:
    full_data = pd.read_csv('full_data.csv')

except:
    nba_player_data = pd.DataFrame(pandas_gbq.read_gbq(player_query,project_id='miscellaneous-projects-444203'))
    team_data = pd.DataFrame(pandas_gbq.read_gbq(team_query,project_id='miscellaneous-projects-444203'))
    opponent_data = team_data.rename(columns={
    col: ('matchup' if col == 'team' else 'game_id' if col == 'game_id' else f'opponent_{col}') for col in team_data.columns})
    full_data = nba_player_data.merge(team_data, on = ['game_id','team'], how = 'inner',suffixes=('','remove'))
    full_data = full_data.merge(opponent_data,on = ['game_id','matchup'],how = 'inner',suffixes=('','remove'))
    full_data.drop([column for column in full_data.columns if 'remove' in column],axis = 1 , inplace=True) 
    full_data.drop([column for column in full_data.columns if '_1' in column],axis = 1 , inplace=True)
    full_data.to_csv('full_data.csv',mode = 'x')

In [ ]:
data_ordered = full_data.sort_values('game_date')

data_ordered.dropna(inplace=True)


#### Feature Engineering Ideas 

* (ratio of 3pa and fga and 3pm and 3pa) TS% for players efg% 
* for players assist_to_turnover ratio assist ratio, 
* rebound_cahnce, defesnive reb %, 
* ast_ratio_season * pace, 
* home * pts season - data pts 3pm avg,
* cold_streak pts_3gm_avg < pts_season boolean, 
* away difficulty away * opponent_defrtg_3gm_avg,
* home_performance = data_ordered[data_ordered["home"] == 1].groupby("team")["pts_season"].mean()
* away_performance = data_ordered[data_ordered["away"] == 1].groupby("team")["pts_season"].mean() these would be to see how the team performance changes 


In [ ]:
data_ordered['pts_per_min_3gm'] = data_ordered['pts_3gm_avg']/data_ordered['min_3gm_avg']
data_ordered['pts_per_min_season'] = data_ordered['pts_season']/data_ordered['min_season']
data_ordered['pts_per_min_momentum'] = data_ordered['pts_per_min_3gm'] - data_ordered['pts_per_min_season']

data_ordered['3pm_per_min_3gm'] = data_ordered['3pm_3gm_avg']/data_ordered['min_3gm_avg']
data_ordered['3pm_per_min_season'] = data_ordered['3pm_season']/data_ordered['min_season']
data_ordered['3pm_per_min_momentum'] = data_ordered['3pm_per_min_3gm'] - data_ordered['3pm_per_min_season'] 

data_ordered['reb_per_min_3gm'] = data_ordered['reb_3gm_avg']/data_ordered['min_3gm_avg']
data_ordered['reb_per_min_season'] = data_ordered['reb_season']/data_ordered['min_season']
data_ordered['reb_per_min_momentum'] = data_ordered['3pm_per_min_3gm'] - data_ordered['reb_per_min_season']

home_performance = data_ordered[data_ordered['home'] == 1]
away_performance = data_ordered[data_ordered['away'] == 1]

In [ ]:
# Ensure data is sorted correctly for chronological calculations
data_ordered = data_ordered.sort_values(by=['player', 'season', 'game_date'])

# Separate home and away games
home_performance = data_ordered[data_ordered['home'] == 1]
away_performance = data_ordered[data_ordered['home'] == 0]  # Fixed to align with `home` flag

# Compute season-to-date averages for home and away games (including game_id)
home_rolling = (
    home_performance.groupby(['player', 'season'])[['game_id', 'pts', 'reb', 'ast', '3pm']]
    .apply(lambda x: x.set_index('game_id').expanding().mean().shift(1))  # Prevent data leakage
    .reset_index()
)

away_rolling = (
    away_performance.groupby(['player', 'season'])[['game_id', 'pts', 'reb', 'ast', '3pm']]
    .apply(lambda x: x.set_index('game_id').expanding().mean().shift(1))
    .reset_index()
)

# Rename columns before merging
home_rolling = home_rolling.rename(columns={'pts': 'home_avg_pts', 'reb': 'home_avg_reb', 
                                            'ast': 'home_avg_ast', '3pm': 'home_avg_3pm'})
away_rolling = away_rolling.rename(columns={'pts': 'away_avg_pts', 'reb': 'away_avg_reb', 
                                            'ast': 'away_avg_ast', '3pm': 'away_avg_3pm'})

# Merge rolling averages back into `data_ordered`
data_ordered = data_ordered.merge(home_rolling[['player', 'game_id', 'home_avg_pts', 'home_avg_reb', 'home_avg_ast', 'home_avg_3pm']],
                                  on=['player', 'game_id'], how='left')

data_ordered = data_ordered.merge(away_rolling[['player', 'game_id', 'away_avg_pts', 'away_avg_reb', 'away_avg_ast', 'away_avg_3pm']],
                                  on=['player', 'game_id'], how='left')

# Fill missing values for early season games
for cat in ['pts', 'reb', 'ast', '3pm']:
    data_ordered[f'home_avg_{cat}'] = data_ordered[f'home_avg_{cat}'].fillna(0)
    data_ordered[f'away_avg_{cat}'] = data_ordered[f'away_avg_{cat}'].fillna(0)

    # Compute home vs. away performance difference conditionally
    data_ordered[f'{cat}_home_away_diff'] = (
        (data_ordered['home'] == 1) * (data_ordered[f'home_avg_{cat}'] - data_ordered[f'away_avg_{cat}']) +
        (data_ordered['home'] == 0) * (data_ordered[f'away_avg_{cat}'] - data_ordered[f'home_avg_{cat}'])
    )

# Drop unnecessary columns
data_ordered = data_ordered.drop(columns=[f'home_avg_{cat}' for cat in ['pts', 'reb', 'ast', '3pm']] + 
                                           [f'away_avg_{cat}' for cat in ['pts', 'reb', 'ast', '3pm']])



In [ ]:
data_ordered.drop(columns = ['Unnamed: 0'],inplace=True)

In [ ]:
data_ordered = data_ordered.groupby(['player','season']).apply(lambda x: x.iloc[3:]).reset_index(drop=True)

In [ ]:
data_ordered.sort_values(by='game_date',inplace=True)

In [ ]:
# WILL APPLY POLYNOMIAL FEATURES IN THE FUTURE
# poly = PolynomialFeatures(2,include_bias=False,interaction_only=False)
# percentages = [feature for feature in data_ordered.columns if '%' in feature]

# for col in percentages:
#     transformed_data = poly.fit_transform(data_ordered[[col]])

#     column_names = [f"{col}_poly_{i+1}" for i in range(transformed_data.shape[1])]

#     poly_df = pd.DataFrame(transformed_data,columns=column_names,index=data_ordered)

# print(poly_df['opponent_ts%_momentum_poly_2'])

In [ ]:
pd.set_option('display.max_columns',100000)

In [ ]:
numeric_columns = data_ordered.select_dtypes(include=['number']).columns.tolist()
numeric_columns = [column for column in numeric_columns if column not in ['pts','reb','ast','blk','stl','3pm','game_id','game_date']]

features = {feature:[] for feature in ['pts','reb','ast','3pm']}

In [ ]:
for category in features.keys():
    print(category)
    for column in numeric_columns:
        correlation = pearsonr(data_ordered[column],data_ordered[category])
        if correlation[1] < .05 and abs(correlation[0]) > .3:
            print(column)
            print(f'correalation {correlation[0]} p_value {correlation[1]}')
            features[category].append(column)

In [ ]:
# Checking spearmanr 

for category in features.keys():
    print(category)
    for column in numeric_columns:
        pearson_corr, pearson_p = pearsonr(data_ordered[column], data_ordered[category])
        spearman_corr, spearman_p = spearmanr(data_ordered[column], data_ordered[category])

        # If Spearman is high but Pearson is low, it suggests a non-linear relationship
        if abs(spearman_corr) > 0.3 and abs(pearson_corr) < 0.2:
            print(f"🚀 {column} likely has a non-linear relationship with {category}")


In [ ]:
#These values appeared to have non-linear relationships applying transformations
data_ordered['ft%_season'] = np.log1p(data_ordered['ft%_season'])
data_ordered['stl_3gm_avg'] = np.log1p(data_ordered['stl_3gm_avg'])
data_ordered['stl_season'] = np.log1p(data_ordered['stl_season'])
data_ordered['to_season'] = data_ordered['to_season']**2 
data_ordered['to_3gm_avg'] = data_ordered['to_3gm_avg']**2 

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

#### SHAP
Applying shap to help reduce collinearity

### Linear Model

In [ ]:
for category in features.keys():
    features[category] = [f for f in features[category] if f != category]


In [ ]:
split_index = int(len(data_ordered) * .80)

train_data = data_ordered.iloc[:split_index]
test_data = data_ordered[split_index:]

for category in features.keys():
    features_list = [f for f in features[category] if f != category]
    x_train,y_train = train_data[features_list],train_data[category]
    x_test, y_test = test_data[features_list],test_data[category]
    linear_model = LinearRegression()

    linear_model.fit(x_train,y_train)

    output = pd.DataFrame({'prediction':linear_model.predict(x_test), 'actual':y_test})
    print(category)
    print(r2_score(y_true=output['actual'],y_pred=output['prediction']))

In [ ]:
#Assessing for multicollinearity
mulitcol_pairs = {cat:[] for cat in features.keys()}
for category in features.keys():
    corr_matrix = data_ordered[features[category]].corr()
    high_corr_vars = np.where(abs(corr_matrix) > 0.8)
    high_corr_pairs = [(corr_matrix.index[x], corr_matrix.columns[y]) 
                    for x, y in zip(*high_corr_vars) if x != y and x < y]
    mulitcol_pairs[category].append(high_corr_pairs)
      # Drop one from each highly correlated pair


In [ ]:
trimmed_feats = {cat:[] for cat in features.keys()}
for cat in mulitcol_pairs.keys():
    print(cat)
    for pairs in mulitcol_pairs[cat]:
        for x,y in pairs:
            cor_x = pearsonr(data_ordered[x],data_ordered[cat])
            cor_y = pearsonr(data_ordered[y],data_ordered[cat])

            if cor_x[1] < .05 and cor_x[0] > cor_y[0]:
                trimmed_feats[cat].append(x)
            elif cor_y[1] < .05 and cor_y[0] > cor_x[0]:
                trimmed_feats[cat].append(y)

In [ ]:
for category in features.keys():
    features_list = [f for f in features[category] if f != category]
    x_train,y_train = train_data[features_list],train_data[category]
    x_test, y_test = test_data[features_list],test_data[category]
    ridge_model = Ridge(alpha=1)

    ridge_model.fit(x_train,y_train)

    output = pd.DataFrame({'prediction':ridge_model.predict(x_test), 'actual':y_test})
    print(category)
    print(r2_score(y_true=output['actual'],y_pred=output['prediction']))

### Random Forrest 

In [ ]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(data_ordered[numeric_columns])

scaled_data_df = pd.DataFrame(scaled_data,columns=numeric_columns)

split_index = int(len(data_ordered) * .80)

scaled_train_data = scaled_data_df.iloc[:split_index]
scaled_test_data = scaled_data_df[split_index:]

In [ ]:
rand_forrest = RandomForestRegressor(n_estimators=100,criterion='squared_error',max_depth=10, min_samples_split=4,n_jobs=-1)

for category in features.keys():
    features_list = [f for f in features[category] if f != category]
    x_train,y_train = scaled_train_data[features_list],train_data[category]
    x_test, y_test = scaled_test_data[features_list],test_data[category]

    rand_forrest.fit(x_train,y_train)

    y_pred = rand_forrest.predict(x_test)

    r2 = r2_score(y_test,y_pred)

    print(category)
    print(r2)


In [ ]:
rand_forrest = RandomForestRegressor(n_estimators=100,criterion='squared_error',max_depth=10, min_samples_split=4,n_jobs=-1)

for category in features.keys():
    features_list = [f for f in features[category] if f != category]
    x_train,y_train = train_data[features_list],train_data[category]
    x_test, y_test = test_data[features_list],test_data[category]

    rand_forrest.fit(x_train,y_train)

    y_pred = rand_forrest.predict(x_test)

    r2 = r2_score(y_test,y_pred)

    print(category)
    print(r2)


### XGboost

In [ ]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(data_ordered[numeric_columns])

scaled_data_df = pd.DataFrame(scaled_data,columns=numeric_columns)

split_index = int(len(data_ordered) * .80)

scaled_train_data = scaled_data_df.iloc[:split_index]
scaled_test_data = scaled_data_df[split_index:]

In [ ]:
param_grid = {'max_depth':[3,6,9],'learning_rate':[.01,.05,.1,.3],'booster':['gbtree','dart'],'subsample':[.5,.7,.9],'colsample_bytree':[.5,.7,.9],'n_estimators':[100,300,500]}
param_linear = {'booster':['gblinear'],'lambda':[0,.1,1,10],'alpha':[0,.1,1,10]}

In [69]:
xgb_regressor = xgboost.XGBRegressor()
mse_score = make_scorer(mean_squared_error,greater_is_better=False)
r2_scorer = make_scorer(r2_score)
scoring = {'MSE':mse_score,'r2':r2_scorer}
grid_search = GridSearchCV(estimator=xgb_regressor,param_grid=param_grid,scoring = scoring,cv=tscv,n_jobs=1,verbose=0,refit='r2')
grid_linear_search = GridSearchCV(estimator=xgb_regressor,param_grid=param_linear,scoring = scoring,cv=tscv,n_jobs=3,verbose=0,refit='r2')


In [ ]:
xg_features = [feature for feature in data_ordered.columns if data_ordered[feature].dtype == 'float' and feature not in features.keys()]

In [ ]:
for category in features.keys():
    x_train,y_train = scaled_train_data[xg_features],train_data[category]
    x_test, y_test = scaled_test_data[xg_features],test_data[category]

    fit_params = {'eval_set':[(x_test,y_test)],'early_stopping_rounds':20,'verbose':False}

    grid_linear_search.estimator.set_params(eval_metric='rmse')


    grid_linear_search.fit(x_train,y_train)


    print(category)
    print(grid_linear_search.best_params_)
    print(grid_linear_search.best_score_)


In [70]:
for category in features.keys():
    x_train,y_train = scaled_train_data[xg_features],train_data[category]
    x_test, y_test = scaled_test_data[xg_features],test_data[category]

    fit_params = {'eval_set':[(x_test,y_test)],'early_stopping_rounds':20,'verbose':False}

    grid_search.estimator.set_params(eval_metric='rmse')


    grid_search.fit(x_train,y_train)

    print(category)
    print(grid_search.best_params_)
    print(grid_search.best_score_)

KeyboardInterrupt: 

### LightGBM

In [ ]:
light = lightgbm()

### Neural Network

In [ ]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(data_ordered[numeric_columns])

scaled_data_df = pd.DataFrame(scaled_data,columns=numeric_columns)

split_index = int(len(data_ordered) * .80)

scaled_train_data = scaled_data_df.iloc[:split_index]
scaled_test_data = scaled_data_df[split_index:]

In [ ]:
param_nn = {'activation':['identity','logistic','tanh','relu'],'solver':['lbfgs','sgd','adam'],'alpha':[.0001,.001,.01,.0005],'batch_size':[200,400,600],'learning_rate':['constant','invscaling','adaptive']}

nn_grid = GridSearchCV(estimator=MLPRegressor(),param_grid=param_nn,scoring=scoring,cv=tscv,n_jobs=-1,verbose=0,refit='r2')


for category in features.keys():
    nn_features = [col for col in scaled_data_df.columns if col not in ['game_id','player','team','matchup','pts','reb','ast','3pm']]

    nn_grid.fit()

### ARIMA   

In [ ]:
arima = ARIMA()

